In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/CMSC723-Final-Project/"

# load the Craigslist Bargains dataset
dataset = load_dataset("craigslist_bargains")

# get train and test parts of the dataset
train_ds = dataset['train']
test_ds = dataset['test']

train_ds[1]

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.53k [00:00<?, ?B/s]

craigslist_bargains.py:   0%|          | 0.00/8.70k [00:00<?, ?B/s]

The repository for craigslist_bargains contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/craigslist_bargains.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5247 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/838 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

{'agent_info': {'Bottomline': ['None', 'None'],
  'Role': ['buyer', 'seller'],
  'Target': [120.0, 200.0]},
 'agent_turn': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 'dialogue_acts': {'intent': ['intro',
   'unknown',
   'inquiry',
   'disagree',
   'init-price',
   'vague-price',
   'counter-price',
   'counter-price',
   'counter-price',
   'agree',
   'agree',
   'offer',
   'accept'],
  'price': [-1.0,
   -1.0,
   -1.0,
   -1.0,
   100.0,
   -1.0,
   120.0,
   150.0,
   145.0,
   -1.0,
   -1.0,
   145.0,
   -1.0]},
 'utterance': ['Hi,I am interested in your board!',
  'Hi, do you have any questions?',
  'Do you know what wood it is made out of?',
  'I do not know specifically but the brand is a gravity skateboard. ',
  'The wheels seem nice on it, but they could be a beter quality. Would you accept 100 for the board?',
  "No, that offer is too low. The board is pretty much brand new as it's been ridden only 4 or 5 times.  The bone bearings are brand new.",
  'If I picked it up at yo

In [ ]:
import uuid

# turn dataset portions into dataframes
train_df = pd.DataFrame(train_ds)
test_df = pd.DataFrame(test_ds)

# give a unique id to every scenario and extract scenario information
def extract_scenario_info(df):
  df_copy = df.copy()
  df.loc[:, 'scenario_id'] = df.apply(lambda _: uuid.uuid4(), axis=1)
  df.loc[:, 'title'] = df['items'].apply(lambda x: x['Title'][0])
  df.loc[:, 'description'] = df['items'].apply(lambda x: x['Description'][0])
  df.loc[:, 'category'] = df['items'].apply(lambda x: x['Category'][0])
  df.loc[:, 'listing_price'] = df['items'].apply(lambda x: x['Price'][0])
  df.loc[:, 'buyer_target'] = df['agent_info'].apply(lambda x: x['Target'][0])
  df.loc[:, 'seller_target'] = df['agent_info'].apply(lambda x: x['Target'][1])

extract_scenario_info(train_df)
extract_scenario_info(test_df)

# collapse intent into dialogue_acts
train_df['dialogue_acts'] = train_df['dialogue_acts'].apply(lambda x: x['intent'])

# filter out empty dialogue acts
train_df = train_df[
    train_df['dialogue_acts'].apply( lambda x: any(act.strip() for act in x) )
]

# create action utterance pairs buyer_kb, seller_kb
train_df.loc[:,'action_utterance_pairs'] = train_df.copy().apply(
    lambda row: [str(f'Action: {act}\nUtterance: {ut}') for act, ut in zip(row['dialogue_acts'], row['utterance'])],
    axis=1
)

# create transcripts from utterances
def create_transcript(agent_no_utterance_pairs):
  transcript = ""
  for agent_no, utterance in agent_no_utterance_pairs:
    agent_name = "Buyer" if agent_no == 0 else "Seller"
    transcript += f" {agent_name}: {utterance}\n"
  return transcript

train_df.loc[:,'transcripts'] = train_df.copy().apply(
    lambda row: create_transcript(zip(row['agent_turn'], row['utterance'])),
    axis=1
)

columns_to_drop = ['items', 'agent_info', 'dialogue_acts', 'agent_turn', 'utterance']
train_df = train_df.drop(columns_to_drop, axis=1)
test_df = test_df.drop(columns_to_drop, axis=1)

In [ ]:
print("Saving training and testing datasets . . .")

# save train and test data to drive
train_df.to_csv(folder_path + 'craigslist_bargains_train.csv', index=False)
test_df.to_csv(folder_path + 'craigslist_bargains_test.csv', index=False)

print("Finished creating training and testing datasets!")

Saving training and testing datasets . . .
Finished creating training and testing datasets!
